In [ ]:
import csv
import json
import sys

csv.field_size_limit(sys.maxsize)

from pathlib import Path

In [ ]:
submission = "packages_midrc-ricord-2021-10-26"
submission_path = Path("/Users/andrewprokhorenkov/CTDS/projects/midrc/midrc_indexing_scripts")
submission_path = submission_path / submission
sequestration_master_file_path = "/Users/andrewprokhorenkov/CTDS/projects/midrc/midrc_indexing_scripts/sequestration/master_sequestration_locations_10162_2022-03-18.tsv"

In [ ]:
seq_master = {}
with open(sequestration_master_file_path) as sequestration_master_file:
    reader = csv.DictReader(sequestration_master_file, delimiter="\t")

    for row in reader:
        seq_master[row["case_ids"]] = row["dataset"]

In [ ]:
package_files = submission_path / "packages"

In [ ]:
def process_package():
    pass

In [ ]:
open_packages = []
seq_packages = []
for package_filepath in package_files.iterdir():
    with open(package_filepath) as package_file:
        reader = csv.DictReader(package_file, delimiter="\t")
        for row in reader:
            item = row

            file_name = item["file_name"]
            case_id = file_name.split("/")[0]
            dataset = seq_master[case_id]

            if dataset == "Open":
                bucket = "open-data-midrc"
                authz = json.dumps(["/programs/Open/projects/R1"])
            elif dataset == "Seq":
                bucket = "sequestered-data-midrc"
                authz = json.dumps(["/programs/SEQ_Open/projects/R3"])
            else:
                print("!")
            item["authz"] = authz
            item["url"] = f"s3://{bucket}/{item['url']}"

            package_contents = json.loads(item["package_contents"].replace("'", "\""))
            for p in package_contents:
                p["size"] = int(p["size"])
            
            item["package_contents"] = json.dumps(package_contents)

            if dataset == "Open":
                open_packages.append(item)
            elif dataset == "Seq":
                seq_packages.append(item)
            else:
                pass

In [ ]:
with open(submission_path / f"packages_open_{submission.split('_')[1]}.tsv", "w") as f:
    fieldnames = [
        "record_type",
        "guid",
        "md5",
        "size",
        "authz",
        "url",
        "file_name",
        "package_contents",
    ]
    writer = csv.DictWriter(f, delimiter="\t", fieldnames=fieldnames)
    writer.writeheader()

    for item in open_packages:
        writer.writerow(item)

with open(submission_path / f"packages_seq_{submission.split('_')[1]}.tsv", "w") as f:
    fieldnames = [
        "record_type",
        "guid",
        "md5",
        "size",
        "authz",
        "url",
        "file_name",
        "package_contents",
    ]
    writer = csv.DictWriter(f, delimiter="\t", fieldnames=fieldnames)
    writer.writeheader()

    for item in seq_packages:
        writer.writerow(item)